<a href="https://colab.research.google.com/github/oksanarev/BIZ_PROJECT/blob/main/BIZ_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Итоговый проект курса "Машинное обучение в бизнесе".

# 1.Обучить модель на понравившихся данных

Загружаем необходимые библиотеки.

In [ ]:
!pip install dill

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score

#working with text
from sklearn.feature_extraction.text import TfidfVectorizer

#normalizing data
from sklearn.preprocessing import StandardScaler

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

С Kaggle загружаем датасет https://www.kaggle.com/datasets/desalegngeb/conversion-predictors-of-cis-to-multiple-sclerosis для классификации перехода клинически изолированного синдрома (CIS) в рассеянный склероз.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 72 bytes


In [ ]:
!kaggle datasets download -d desalegngeb/conversion-predictors-of-cis-to-multiple-sclerosis

conversion-predictors-of-cis-to-multiple-sclerosis.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip conversion-predictors-of-cis-to-multiple-sclerosis.zip

Archive:  conversion-predictors-of-cis-to-multiple-sclerosis.zip
replace conversion_predictors_of_clinically_isolated_syndrome_to_multiple_sclerosis.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
df = pd.read_csv('conversion_predictors_of_clinically_isolated_syndrome_to_multiple_sclerosis.csv')
df.head(5)

,Unnamed: 0,Gender,Age,Schooling,Breastfeeding,Varicella,Initial_Symptom,Mono_or_Polysymptomatic,Oligoclonal_Bands,LLSSEP,ULSSEP,VEP,BAEP,Periventricular_MRI,Cortical_MRI,Infratentorial_MRI,Spinal_Cord_MRI,Initial_EDSS,Final_EDSS,group
0,0,1,34,20.0,1,1,2.0,1,0,1,1,0,0,0,1,0,1,1.0,1.0,1
1,1,1,61,25.0,3,2,10.0,2,1,1,0,1,0,0,0,0,1,2.0,2.0,1
2,2,1,22,20.0,3,1,3.0,1,1,0,0,0,0,0,1,0,0,1.0,1.0,1
3,3,2,41,15.0,1,1,7.0,2,1,0,1,1,0,1,1,0,0,1.0,1.0,1
4,4,2,34,20.0,2,1,6.0,2,0,1,0,0,0,1,0,0,0,1.0,1.0,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               273 non-null    int64  
 1   Gender                   273 non-null    int64  
 2   Age                      273 non-null    int64  
 3   Schooling                272 non-null    float64
 4   Breastfeeding            273 non-null    int64  
 5   Varicella                273 non-null    int64  
 6   Initial_Symptom          272 non-null    float64
 7   Mono_or_Polysymptomatic  273 non-null    int64  
 8   Oligoclonal_Bands        273 non-null    int64  
 9   LLSSEP                   273 non-null    int64  
 10  ULSSEP                   273 non-null    int64  
 11  VEP                      273 non-null    int64  
 12  BAEP                     273 non-null    int64  
 13  Periventricular_MRI      273 non-null    int64  
 14  Cortical_MRI             2

In [ ]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df.columns

Index(['Gender', 'Age', 'Schooling', 'Breastfeeding', 'Varicella',
       'Initial_Symptom', 'Mono_or_Polysymptomatic', 'Oligoclonal_Bands',
       'LLSSEP', 'ULSSEP', 'VEP', 'BAEP', 'Periventricular_MRI',
       'Cortical_MRI', 'Infratentorial_MRI', 'Spinal_Cord_MRI', 'Initial_EDSS',
       'Final_EDSS', 'group'],
      dtype='object')

In [ ]:
df=df[df['Initial_Symptom'].isna()==False]
df=df[df['Schooling'].isna()==False]

In [ ]:
df.Schooling.value_counts()

20.0    83
15.0    81
12.0    58
9.0     34
22.0     4
6.0      3
25.0     2
14.0     2
0.0      1
23.0     1
24.0     1
8.0      1
Name: Schooling, dtype: int64

In [ ]:
df['Initial_EDSS'].fillna(method='ffill', inplace=True)
df['Final_EDSS'].fillna(method='ffill', inplace=True)

In [ ]:
df.shape

(271, 19)

In [ ]:
df['Age'] = df['Age'].astype (int).astype (float)

In [ ]:
df.Age.dtype

dtype('float64')

In [ ]:
df.loc[df['group']== 2, 'group'] = 0

In [ ]:
df.group.value_counts()

0    146
1    125
Name: group, dtype: int64

In [ ]:
X = df.drop(['group'],axis=1)
y = df['group']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

# save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

# save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

Для классификации выбираем модель GradientBoostingClassifier и перед запуском модели проводим исследование важности признаков, чтобы не брать все 19 (классификатор на всех признаках выдавал хороший результат, но после исключения части признаков он сработал не очень, возможно для интеграции с клиентом надо было прописать все признаки).

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.feature_selection import RFE

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
rfe = RFE(estimator=GradientBoostingClassifier(), n_features_to_select=6)

In [ ]:
model = GradientBoostingClassifier()

In [ ]:
pipe = Pipeline([('Feature Selection', rfe), ('Model', model)])
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)
n_scores = cross_val_score(pipe, X_train, y_train, scoring='precision_macro')
np.mean(n_scores)

0.7671688144941612

In [ ]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('Feature Selection',
                 RFE(estimator=GradientBoostingClassifier(),
                     n_features_to_select=6)),
                ('Model', GradientBoostingClassifier())])

In [ ]:
rfe.support_

array([False,  True,  True, False, False,  True, False, False, False,
       False, False, False,  True, False,  True, False, False,  True])

In [ ]:
pd.DataFrame(rfe.support_,index=X.columns,columns=['Rank'])

,Rank
Gender,False
Age,True
Schooling,True
Breastfeeding,False
Varicella,False
Initial_Symptom,True
Mono_or_Polysymptomatic,False
Oligoclonal_Bands,False
LLSSEP,False
ULSSEP,False


In [ ]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]

class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [ ]:
X_train.head()

,Gender,Age,Schooling,Breastfeeding,Varicella,Initial_Symptom,Mono_or_Polysymptomatic,Oligoclonal_Bands,LLSSEP,ULSSEP,VEP,BAEP,Periventricular_MRI,Cortical_MRI,Infratentorial_MRI,Spinal_Cord_MRI,Initial_EDSS,Final_EDSS
75,1,43.0,12.0,2,2,8.0,2,0,1,1,1,0,1,1,1,0,2.0,2.0
116,1,23.0,20.0,1,1,8.0,2,1,1,1,1,0,1,1,0,0,2.0,2.0
134,2,21.0,20.0,2,2,8.0,2,0,0,0,0,0,0,0,0,0,1.0,1.0
158,1,27.0,20.0,1,1,1.0,2,0,0,0,0,0,1,0,0,0,1.0,1.0
188,2,24.0,20.0,3,1,3.0,2,0,0,0,0,0,1,0,0,1,1.0,1.0


Выбрали 6 признаков.

In [ ]:
categorical_columns = ['Periventricular_MRI', 'Infratentorial_MRI', 'Final_EDSS']
continuous_columns = ['Age', 'Initial_Symptom', 'Schooling']

In [ ]:
Periventricular_MRI = Pipeline([
                ('selector', FeatureSelector('Periventricular_MRI')),
                ('ohe', OHEEncoder(key='Periventricular_MRI'))
            ])
Infratentorial_MRI = Pipeline([
                ('selector', FeatureSelector('Infratentorial_MRI')),
                ('ohe', OHEEncoder(key='Infratentorial_MRI'))
            ])
Final_EDSS = Pipeline([
                ('selector', FeatureSelector('Final_EDSS')),
                ('ohe', OHEEncoder(key='Final_EDSS'))
            ])
Age = Pipeline([
                ('selector', NumberSelector(key='Age'))
            ])
Initial_Symptom = Pipeline([
                ('selector', NumberSelector(key='Initial_Symptom'))
            ])
Schooling = Pipeline([
                ('selector', NumberSelector(key='Schooling'))
            ])

In [ ]:
#feats = FeatureUnion(final_transformers)
feats = FeatureUnion([('Periventricular_MRI', Periventricular_MRI),
                      ('Infratentorial_MRI', Infratentorial_MRI),
                      ('Final_EDSS', Final_EDSS),
                      ('Age', Age),
                      ('Initial_Symptom', Initial_Symptom),
                      ('Schooling', Schooling)])

feature_processing = Pipeline([('feats', feats)])

In [ ]:
pipeline = Pipeline([
    ('features',feats),
    ('classifier',GradientBoostingClassifier()),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Periventricular_MRI',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Periventricular_MRI')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Periventricular_MRI'))])),
                                                ('Infratentorial_MRI',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Infratentorial_MRI')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Infratentorial_MRI'))])),...
                                                                  FeatureSelector(column='Final_EDSS')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Final_EDSS'))])),
                                                ('Age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Age'))])),
                                                ('Initial_Symptom',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Initial_Symptom'))])),
                                                ('Schooling',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Schooling'))]))])),
                ('classifier', GradientBoostingClassifier())])

In [ ]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('Periventricular_MRI',
                                  Pipeline(steps=[('selector',
                                                   FeatureSelector(column='Periventricular_MRI')),
                                                  ('ohe',
                                                   OHEEncoder(key='Periventricular_MRI'))])),
                                 ('Infratentorial_MRI',
                                  Pipeline(steps=[('selector',
                                                   FeatureSelector(column='Infratentorial_MRI')),
                                                  ('ohe',
                                                   OHEEncoder(key='Infratentorial_MRI'))])),
                                 ('Final_EDSS',
                                  Pipeline(steps=[('selector',
                                                   FeatureSelector(column='Final_EDSS')),
                                              

In [ ]:
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

In [ ]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [ ]:
X_test.head(3)

,Gender,Age,Schooling,Breastfeeding,Varicella,Initial_Symptom,Mono_or_Polysymptomatic,Oligoclonal_Bands,LLSSEP,ULSSEP,VEP,BAEP,Periventricular_MRI,Cortical_MRI,Infratentorial_MRI,Spinal_Cord_MRI,Initial_EDSS,Final_EDSS
0,2,48.0,12.0,1,1,14.0,2,1,0,0,0,0,1,1,0,0,2.0,2.0
1,2,37.0,20.0,1,1,8.0,2,0,1,1,0,0,1,0,0,0,1.0,1.0
2,2,43.0,15.0,1,2,5.0,2,0,1,1,0,0,0,0,0,0,1.0,1.0


In [ ]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [ ]:
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Periventricular_MRI',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Periventricular_MRI')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Periventricular_MRI'))])),
                                                ('Infratentorial_MRI',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Infratentorial_MRI')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Infratentorial_MRI'))])),...
                                                                  FeatureSelector(column='Final_EDSS')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Final_EDSS'))])),
                                                ('Age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Age'))])),
                                                ('Initial_Symptom',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Initial_Symptom'))])),
                                                ('Schooling',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Schooling'))]))])),
                ('classifier', GradientBoostingClassifier())])

In [ ]:
preds = pipeline.predict_proba(X_test)[:, 1]

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix],
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.703609, F-Score=0.758, Precision=0.862, Recall=0.676


# 2. Создать rest api сервис, к которому можно будет обращаться для получения прогнозов

Создаём облачный сервер на основе **ngrok**.

In [ ]:
!pip install ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!./ngrok authtoken 25vEpcJ5Ih4vlUp4thEZ9sEA6ZU_3Bnu17gKacRXhF6hLeefc

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
cat /root/.ngrok2/ngrok.yml

authtoken: 25vEpcJ5Ih4vlUp4thEZ9sEA6ZU_3Bnu17gKacRXhF6hLeefc


In [ ]:
# Пробный запуск Flask

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/a")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://0d55-35-190-174-155.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


In [ ]:
# Загружаем обученные модели
with open('logreg_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [ ]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [ ]:
# Обработчики и запуск Flask
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])
def general():
    return "Добро пожаловать в сервис диагностики рассеенного склероза:"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    Periventricular_MRI, Infratentorial_MRI, Final_EDSS, Age, Initial_Symptom, Schooling= "", "", "", "", "", ""
    request_json = request.get_json()

    if request_json["Periventricular_MRI"]:
        Periventricular_MRI = request_json['Periventricular_MRI']

    if request_json["Infratentorial_MRI"]:
        Infratentorial_MRI = request_json['Infratentorial_MRI']

    if request_json["Final_EDSS"]:
        Final_EDSS = request_json['Final_EDSS']

    if request_json["Age"]:
        Age = request_json['Age']

    if request_json["Initial_Symptom"]:
        Initial_Symptom = request_json['Initial_Symptom']

    if request_json["Schooling"]:
        Schooling = request_json['Schooling']

    print (Periventricular_MRI)
    preds = model.predict_proba(pd.DataFrame({"Periventricular_MRI": [Periventricular_MRI],
                                            "Infratentorial_MRI": [Infratentorial_MRI],
                                            "Final_EDSS": [Final_EDSS],
                                            "Age": [Age],
                                            "Initial_Symptom": [Initial_Symptom],
                                            "Schooling": [Schooling]
                                              }))
    data["predictions"] = preds[:, 1][0]
    data["Periventricular_MRI"] = Periventricular_MRI
        # indicate that the request was a success
    data["success"] = True
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://87cb-35-190-174-155.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [14/Jun/2023 13:51:01] "POST /predict HTTP/1.1" 200 -


1.0
OK


...

1.0
OK

OK


INFO:werkzeug:127.0.0.1 - - [14/Jun/2023 13:51:16] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Jun/2023 13:51:16] "POST /predict HTTP/1.1" 200 -


1.0
OK

OK


INFO:werkzeug:127.0.0.1 - - [14/Jun/2023 13:51:16] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Jun/2023 13:51:17] "POST /predict HTTP/1.1" 200 -


1.0
OK
1.0
OK
